# Welcome to the Kitchen Aide Agent 🐝


🎯 Scenario: You need some help figuring out what to make with the food in your fridge and what you might need to buy diring your next grocery shop.

Your Kitchen-Aide Agent can assist you by checking to see what foods are in your pantry, regrigerator, and freezer. It also has access to all your personal recipes as well as new ones online.

You can ask Kitchen-Aide questions like:
- Which ingedients am I missing to create X recipe? Add them to my shopping list
- What can I make with my ingredients on hand?
- What food do I have on hand?

## 🔧 Setup
First, let's install the BeeAI Framework and set up our environment.

- setting up the observability so we can capture and log the actions our agent takes
- getting the "internal documents"

In [ ]:
%pip install -Uqq arize-phoenix s3fs unstructured "requests==2.32.4"\
 "openinference-instrumentation-beeai==0.1.13" \
 "beeai-framework[duckduckgo,rag]" "fsspec==2025.3.0" jedi \
 "opentelemetry-api==1.37.0" "opentelemetry-sdk==1.37.0" "langgraph<=0.5.0"

# The following wraps Notebook output
from IPython.display import HTML, display
def set_css(*_, **__):
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))
get_ipython().events.register("pre_run_cell", set_css)

Now let's import the necessary modules:


In [ ]:
import os
import asyncio
import time
import phoenix as px
import ipywidgets
from typing import Any, Optional
from datetime import date
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from beeai_framework.agents.requirement import RequirementAgent
from beeai_framework.agents.requirement.types import RequirementAgentOutput
from beeai_framework.agents.requirement.requirements import Requirement, Rule
from beeai_framework.agents.requirement.requirements.conditional import ConditionalRequirement
from beeai_framework.backend import ChatModel, ChatModelParameters
from beeai_framework.backend.document_loader import DocumentLoader
from beeai_framework.backend.embedding import EmbeddingModel
from beeai_framework.backend.text_splitter import TextSplitter
from beeai_framework.backend.vector_store import VectorStore
from beeai_framework.context import RunContext
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory import UnconstrainedMemory
from beeai_framework.middleware.trajectory import GlobalTrajectoryMiddleware
from beeai_framework.tools import Tool, ToolRunOptions, tool, StringToolOutput
from beeai_framework.tools.search.duckduckgo import DuckDuckGoSearchTool
from beeai_framework.tools.search.retrieval import VectorStoreSearchTool
from beeai_framework.tools.think import ThinkTool
from beeai_framework.tools.weather import OpenMeteoTool
from beeai_framework.tools.types import ToolRunOptions
from openinference.instrumentation.beeai import BeeAIInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

 ## 1️⃣ LLM Providers: Choose Your AI Engine

BeeAI Framework supports 10+ LLM providers including Ollama, Groq, OpenAI, Watsonx.ai, and more, giving you flexibility to choose local or hosted models based on your needs. You can find the documentation on how to connect to other providers [here](https://framework.beeai.dev/modules/backend).


### *❗* Exercise: Select your Language Model Provider

Change the `provider` and `model` variables to your desired provider and model.

If you select a provider that requires an API key URL or Project_ID, select the key icon on the left menu and set the variables to match those in the userdata.get() function.

Try several models to see how your agent performs. Note that you may need to modify the system prompt for each model, as they all have their own system prompt best practice.

In [ ]:
#Use widgets to show provider choices
providers=ipywidgets.ToggleButtons(options=['ollama','openai'])
display(providers)

In Colab, install and start Ollama for providing the Embedding Model used during RAG operations.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

In [ ]:
provider=providers.value
from google.colab import userdata
# Ollama - No parameters required
if provider=="ollama":
    model="granite4:tiny-h"
    #model="granite3.3"
    provider_model=provider+":"+model
    !ollama pull $model
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=0))
# OpenAI - Place OpenAI API Key in Colab Secrets (key icon) as OPENAI_KEY
elif provider=="openai":
    model="gpt-5-mini"
    provider_model=provider+":"+model
    api_key = userdata.get('OPENAI_KEY')             #Set secret value using key in left menu
    llm=ChatModel.from_name(provider_model, ChatModelParameters(temperature=1), api_key=api_key)
else:
  print("Provider " + provider + " undefined")

# Pull Recipe and Food Stock Data into Colab

To delete when confirmed working

In [ ]:
import os

if os.getcwd() == "/":
    os.chdir('/content/')

if os.path.exists("beeai-workshop"):
    !rm -rf beeai-workshop

# Sparse checkout only kitchen-contents directory
!git clone --filter=blob:none --no-checkout https://github.com/IBM/beeai-workshop.git
os.chdir("beeai-workshop")
!git sparse-checkout init --cone
!git sparse-checkout set kitchen-aide/kitchen-contents
!git checkout

os.chdir("kitchen-aide/kitchen-contents")

print("Now inside:", os.getcwd())


In [ ]:
folder = "/content/beeai-workshop/kitchen-aide/kitchen-contents"
num_lines_to_show = 20   # ← change this to show more or fewer lines

for fname in sorted(os.listdir(folder)):
    fpath = os.path.join(folder, fname)

    if os.path.isfile(fpath):
        print("\n" + "="*80)
        print(f"📄 {fname}  —  showing first {num_lines_to_show} lines")
        print("="*80)

        with open(fpath, "r") as f:
            for i in range(num_lines_to_show):
                line = f.readline()
                if not line:
                    break
                print(line.rstrip())


## 2️⃣ Provide System Prompt

### *❗* Exercise: Customize Your System Prompt
Try modifying the system prompt. Customize the "basic rules" section to add your own. Note that changes to the system prompt will affect the performance of the model. Creating a great `System Prompt` is an art, not a science.

In [ ]:
todays_date = date.today().strftime("%B %d, %Y")
instruct_prompt = f"""You help to assess ingredients needed to prepare recipes

Today's date is {todays_date}.

Tools:
- ThinkTool: Helps you plan and reason before you act. Use this tool when you need to think.
- DuckDuckGoSearchTool: Use this tool to search for recipes on the internet
- get_pantry_tool: Use this tool to gather a list of your ingredients in your pantry.
- get_fridge_tool: Use this tool to gather a list of your ingredients in your fridge.
- get_freezer_tool: Use this tool to gather a list of your ingredients in your freezer.
- my_recipe_search: your personal recipes.

Basic Rules:
- Determine the recipes that can be prepared with the ingredients that we have in our pantry, fridge or freezer
- Favor my recipes over recipes on the internet
- If we cant' prepare a recipe, make a shopping list of the items needed for a recipe
"""

## 3️⃣ Memory Systems: Maintaining Context across iterations or sessions

In [ ]:
from beeai_framework.memory import UnconstrainedMemory

memory = UnconstrainedMemory()

## 4️⃣ Tools: Enabling LLMs to Take Action

The BeeAI framework provides [built in tools](https://framework.beeai.dev/modules/tools#built-in-tools) for common tool types, but also provides the ability to create [custom tools](https://framework.beeai.dev/modules/tools#creating-custom-tools).

### Adding Framework Provided Tools
The **Think tool** encourages a Re-Act or planning pattern (depending on how it is used) where the agent reasons and plans before calling a tool.

The DuckDuckGoSearchTool is a Web Search tool that provides relevant data from the internet to the LLM

In [ ]:
think_tool = ThinkTool()
internet_search_tool = DuckDuckGoSearchTool()

### Add Custom Tool to Read Kitchen Inventory


There are 2 ways to provide custom tools to your agent. For simple tools you can use the `@tool` decorator above the function. For more complex tools, you can extend the `Tool Class` and customize things such as the run time and tool execution.  

We will extend the tool class in this example. To learn more about advanced tool customization, take a look at this section in the [documentation](https://framework.beeai.dev/modules/tools#advanced-custom-tool).

In [ ]:
class InventoryToolInput(BaseModel):
    pass


class InventoryTool(Tool[InventoryToolInput, ToolRunOptions, StringToolOutput]):
    """
    Base inventory tool. Subclasses define the location and tool name.
    """

    # These will be overridden in subclasses:
    name = "InventoryBase"
    description = "Base inventory tool – do not register directly."
    input_schema = InventoryToolInput

    def __init__(
        self,
        location: str,
        options: dict[str, Any] | None = None,
    ) -> None:
        self.location = location
        super().__init__(options)

    def _create_emitter(self) -> Emitter:
        return Emitter.root().child(
            namespace=["tool", "example", "result"],
            creator=self,
        )

    async def _run(
        self, input: InventoryToolInput, options: ToolRunOptions | None, context: RunContext
    ) -> StringToolOutput:
        # Use the fixed location for this tool instance
        location = self.location
        file_path = f"{location}_contents.md"

        try:
            with open(file_path, "r") as file:
                result = file.read()
        except Exception as e:
            # You might want to return an error string instead of printing
            result = f"Error reading file {file_path}: {e}"

        print(location, "inventory:", result)
        return StringToolOutput(result=result)


class PantryInventoryTool(InventoryTool):
    name = "get_pantry_inventory"
    description = "Search the pantry inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="pantry", options=options)


class FreezerInventoryTool(InventoryTool):
    name = "get_freezer_inventory"
    description = "Search the freezer inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="freezer", options=options)


class FridgeInventoryTool(InventoryTool):
    name = "get_fridge_inventory"
    description = "Search the fridge inventory for ingredients."

    def __init__(self, options: dict[str, Any] | None = None):
        super().__init__(location="fridge", options=options)


#Three distinct tools as far as the framework is concerned
get_pantry_tool = PantryInventoryTool()
get_freezer_tool = FreezerInventoryTool()
get_fridge_tool = FridgeInventoryTool()


## 5️⃣ Creating a RAG (Retrieval Augmented Generation) Tool to Search My Recipes

The BeeAI Framework has built in abstractions to make RAG simple to implement. Read more about it [here](https://framework.beeai.dev/modules/rag).

First, we must pull an embedding model which converts text into numerical vectors so we can compare meanings and retrieve the most relevant snippets. The original document is:
1. preprocessed (cleaned + broken into chunks)
2. ran through the embedding algorithm
3. stored in the vector database (in system RAM for thi example)


In [ ]:
!ollama pull nomic-embed-text:latest

In [ ]:
embedding_model = EmbeddingModel.from_name("ollama:nomic-embed-text")

### *❗* Exercise: Internal documents
Take a look at the internal documents so you know what type of questions to ask your agent


In [ ]:
file_path="my_recipes.md"
with open(file_path, 'r') as file:
    content = file.read()

#Print first 50 lines
lines = content.splitlines()
for i in range(min(50, len(lines))):
    print(lines[i])

In [ ]:
from beeai_framework.backend.types import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on=[("#", "Header 1"), ("##", "Header 2")]

text_splitter = TextSplitter.from_name(
    name="langchain:MarkdownHeaderTextSplitter",
    headers_to_split_on=headers_to_split_on, strip_headers = False
    )
doc_splits = await text_splitter.split_text(content)

# Convert each Langchain Document to a BeeAI Framework Document
documents = [
    Document(content=split.page_content, metadata=split.metadata)
    for split in doc_splits
]

# Print the first document to verify
##print(documents[0])

Create the `TemporalVectorStore`, which means this vector store also tracks time.

In [ ]:
# Create vector store and add documents
vector_store = VectorStore.from_name(name="beeai:TemporalVectorStore", embedding_model=embedding_model)
await vector_store.add_documents(documents=documents)
print("Vector store populated with documents")

Create the `internal_document_search` tool! Because the `VectorStoreSearchTool` is a built in tool wrapper, we don't need to use the `@tool` decorator or extend the custom `Tool class`.

In [ ]:
# Create the vector store search tool
my_recipe_search = VectorStoreSearchTool(vector_store=vector_store)

## Explore Observability: See what is happening under the hood

Create the function that sets up observability using `OpenTelemetry` and [Arize's Phoenix Platform](https://arize.com/docs/phoenix/inferences/how-to-inferences/manage-the-app). There a several ways to view what is happening under the hood of your agent. View the observability documentation [here](https://framework.beeai.dev/modules/observability).

In [ ]:
def setup_observability(endpoint: str = "http://localhost:6006/v1/traces") -> None:
    """
    Sets up OpenTelemetry with OTLP HTTP exporter and instruments the beeai framework.
    """
    resource = Resource(attributes={})
    tracer_provider = trace_sdk.TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
    trace_api.set_tracer_provider(tracer_provider)

    BeeAIInstrumentor().instrument()

In [ ]:
load_dotenv()
# Enable OpenTelemetry integration
setup_observability("http://localhost:6006/v1/traces")
px_session = px.launch_app()

## 6️⃣ Conditional Requirements: Guiding Agent Behavior


What Are Conditional Requirements?
[Conditional requirements](https://framework.beeai.dev/experimental/requirement-agent#conditional-requirement) ensure your agents are reliable by controlling when and how tools are used. They're like business rules for agent behavior. You can make them as strict (esentially writing a static workflow) or flexible (no rules! LLM decides) as you'd like.

The rules that you enforce may seem simple in the BeeAI framework, but in other frameworks they require ~5X the amount of code. Check out this [blog](https://beeai.dev/blog/reliable-ai-agents) where we built the same agent in BeeAI and other agent framework LangGraph.

These conditional requirements enforce the following in only 6 lines of code:
1. The agent must call the think tool as the first tool call and then after every other tool enforcing a re-act pattern. It is not allowed to call the think tool consecutive times in a row.
3. The DuckDuckGo Internet search tool can also only be called after the my_recipe_search tool, it is allowed to be called up to 3 times, and it has a relative priority of 15.
4. The get_fridge_tool tool must be called before the get_freezer_tool as things in the fridge go bad quicker. It can only be called once.
5. The



##  7️⃣ Assemble Your Reliable BeeAI Agent

This is the part we've been working towards! Let's assemble the agent with all the parts we created.

NEED TO ADD BACK IN MY_RECIPE SEARCH ONCE RAG IS FIXED. AND NEED TO ADD BACK IN THE ONLY AFTER CONDITION FOR DUCK DUCK GO

In [ ]:
agent = RequirementAgent(
    llm=llm,
    instructions= instruct_prompt,
    memory = memory,
    tools=[think_tool, internet_search_tool, get_pantry_tool, get_freezer_tool, get_fridge_tool],
    requirements=[
        ConditionalRequirement(think_tool, consecutive_allowed=False, force_at_step=1 ),
        ConditionalRequirement(get_pantry_tool, only_after=[get_fridge_tool,think_tool], min_invocations=1, priority=10),
        ConditionalRequirement(get_freezer_tool, only_after=[get_fridge_tool,think_tool], min_invocations=1, priority=5 ),
        ConditionalRequirement(get_fridge_tool, min_invocations=1, priority=15 ),
        #ConditionalRequirement(my_recipe_search),
        ConditionalRequirement(internet_search_tool, max_invocations=3, only_after= [get_fridge_tool,think_tool]),
    ],
    # Log intermediate steps to the console
    middlewares=[GlobalTrajectoryMiddleware(included=[Tool])],
)

### *❗* Exercise: Test Your Agent
Change the execution settings and see what happens. Does your agent run out of iterations? Every task is different and its important to balance flexibility with control.

Example Questions:
- Which of my recipes can I prepare with the items I have on hand?
- What do I need to buy to make blueberry scones?
- What breakfast recipes can I make with the items I have on hand?
- What internet recipe can I prepare with the items I have on hand?
- What items do I need to buy to prepare my pizza recipe?

In [ ]:
while (question := input("How can Kitchen-Aide Help you? (Enter q to end): ")) !="q":
    response = await agent.run(question, max_retries_per_step=3, total_max_retries=25)
    print(response.last_message.text)

In [ ]:
px_session.view()